Limpieza de datos Rooms

In [508]:
import pandas as pd
import numpy as np
import re 

In [509]:
data_location = "./Data/properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [510]:
data["rooms_completo"] =data["rooms"]
data_room_null_description = data.description.astype(str).str.lower()
data_room_null_title = data.title.astype(str).str.lower()
display(data_room_null_description)
display(data_room_null_title)

0         2 ambientes tipo casa planta baja por pasillo,...
1         venta de departamento en décimo piso al frente...
2         2 ambientes 3er piso lateral living comedor am...
3         ph 3 ambientes con patio. hay 3 deptos en lote...
4         departamento con fantástica iluminación natura...
                                ...                        
121215    torre forum alcorta - máxima categoría.impecab...
121216    excelente e impecable casa en venta en las lom...
121217    venta departamento ambiente divisible a estren...
121218    2 amb al contrafrente,  luminoso. el departame...
121219    departamento de 77 m2 apto profesional. edific...
Name: description, Length: 121220, dtype: object

0                        2 amb tipo casa sin expensas en pb
1         venta depto 2 dorm. a estrenar 7 e/ 36 y 37   ...
2                  2 amb 3er piso con ascensor apto credito
3                              ph 3 amb. cfte. reciclado   
4         depto 2 amb al contrafrente zona centro/plaza ...
                                ...                        
121215           torre forum alcorta- impecable 3 ambientes
121216    ruca inmuebles | venta | lomas de san isidro |...
121217    venta departamento ambiente divisible a estren...
121218    2 amb. c/ dep. de servicio al contrafrente| re...
121219           departamento apto profesional en el centro
Name: title, Length: 121220, dtype: object

In [511]:
print("Primer grupo de filas sin rooms")
data["rooms_completo"].isnull().value_counts()

Primer grupo de filas sin rooms


True     73830
False    47390
Name: rooms_completo, dtype: int64

Seteamos los que son monoambientes
Primero buscamos las palabras que identifiquen al monoambiente. 
Luego buscamos las coincidencias dentro del universo de coincidencias del campo descripción
donde hubo coincidencias, le asignamos 1. 

In [512]:
def fun_buscar_patt(data_validar,patt_room):
    patt_regex = re.compile(patt_room)
    data_match = data_validar.apply(lambda x:  x if x is np.NaN else patt_regex.search(x))
    return data_match

In [513]:
def fun_buscar_disminuir(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar)).astype(int)
    data.loc[room_null_mask,"regex"] = patt
    print("Disminución en función a los ambientes tipo:",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [543]:
def fun_buscar_disminuir_mapper(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
       'un': 1,'uun':1, 'uno': 1,'dos': 2,'tres': 3,'cuatro':4,
        'cinco':5,'sinco':5,'seis':6,'ceis':6,'siete':7,
        'ciete':7,'ocho':8,'nueve':9,'diez':10,'once':11,
        'doce':12,'trece':13,'catorce':14,'quince':15,
        'dieciseis':16,'diecisiete':17,'dieciocho':18,'diecinueve':19,
        'veinte':20
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    data.loc[room_null_mask,"regex"] = patt
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [515]:
#validar que luego elija siempre la última que encuentre

In [516]:
def fun_buscar_disminuir_mapper_2(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
                'segundo': 2,'tercer':3, 'cuarto': 4,'quinto': 5,'sexto': 6
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    data.loc[room_null_mask,"regex"] = patt
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [517]:
def disminuciones_repeticion(serie,patt):
    print(data["rooms_completo"].isnull().value_counts())
    data_encontrada = serie.apply(lambda x: x if x is np.NaN else x.count(patt))
    maskarita= data_encontrada>0|data["rooms_completo"].isnull()
    data.loc[maskarita,"rooms_completo"] = data_encontrada[maskarita]
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [518]:
data_room_match = fun_buscar_patt(data_room_null_description,r"(mono ambiente|monoambiente|mono-ambiente|Monoamb)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
data.loc[data_room_match_mask,"regex"] = "(mono ambiente|monoambiente|mono-ambiente)"
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

Primera disminución Monoambientes


True     69080
False    52140
Name: rooms_completo, dtype: int64

In [519]:
data_room_match = fun_buscar_patt(data_room_null_title,r"(mono ambiente|monoambiente|mono-ambiente|monoamb)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
data.loc[data_room_match_mask,"regex"] = "(mono ambiente|monoambiente|mono-ambiente|monoamb)"
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

Primera disminución Monoambientes


True     68675
False    52545
Name: rooms_completo, dtype: int64

In [520]:
resultado = fun_buscar_disminuir(data_room_null_description,r"[\s](\d\d)[\s]?habitaci",0)
print(resultado)

Disminución en función a los ambientes tipo: [\s](\d\d)[\s]?habitaci
True     68631
False    52589
Name: rooms_completo, dtype: int64


In [521]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+)[\s]?(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(dormitorio|domitorio)
False    64513
True     56707
Name: rooms_completo, dtype: int64


In [522]:
resultado = fun_buscar_disminuir_mapper_2(data_room_null_description,r"(\w+)[\s]?(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(dormitorio|domitorio)
False    65214
True     56006
Name: rooms_completo, dtype: int64


In [523]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+)[\s]?(ambiente)",1)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(ambiente)
False    67069
True     54151
Name: rooms_completo, dtype: int64


In [524]:
resultado = fun_buscar_disminuir(data_room_null_description,r"[\s](\d)[\s]?(amb)",1)
print(resultado)

Disminución en función a los ambientes tipo: [\s](\d)[\s]?(amb)
False    82384
True     38836
Name: rooms_completo, dtype: int64


In [525]:
resultado = fun_buscar_disminuir(data_room_null_description,r"\D(\d)[\s]?(dorm)",0)
print(resultado)

Disminución en función a los ambientes tipo: \D(\d)[\s]?(dorm)
False    100893
True      20327
Name: rooms_completo, dtype: int64


In [526]:
resultado = fun_buscar_disminuir(data_room_null_title,r"\D(\d?\d)[\s]?(dorm)",0)
print(resultado)

Disminución en función a los ambientes tipo: \D(\d?\d)[\s]?(dorm)
False    101365
True      19855
Name: rooms_completo, dtype: int64


In [527]:
resultado = fun_buscar_disminuir(data_room_null_description,r"(\d)[\s]?(suite)",0)
print(resultado)

Disminución en función a los ambientes tipo: (\d)[\s]?(suite)
False    101515
True      19705
Name: rooms_completo, dtype: int64


Buscamos ahora las restantes en el título

In [557]:
resultado = fun_buscar_disminuir(data_room_null_title,r"([\s]\d?\d)[\s]?amb",1)
print(resultado)

Disminución en función a los ambientes tipo: ([\s]\d?\d)[\s]?amb
False    121219
True          1
Name: rooms_completo, dtype: int64


In [529]:
resultado = fun_buscar_disminuir_mapper(data_room_null_title,r"(\w+)[\s]?(amb)",1)
print(resultado)

Disminución en función a los ambientes : (\w+)[\s]?(amb)
False    102857
True      18363
Name: rooms_completo, dtype: int64


In [530]:
disminuciones_repeticion(data_room_null_description,"dormitorio")

False    102857
True      18363
Name: rooms_completo, dtype: int64


False    104745
True      16475
Name: rooms_completo, dtype: int64

In [533]:
disminuciones_repeticion(data_room_null_description,"habitacion")


False    105019
True      16201
Name: rooms_completo, dtype: int64


False    105122
True      16098
Name: rooms_completo, dtype: int64

In [504]:
disminuciones_repeticion(data_room_null_description,"suite")

False    105034
True      16186
Name: rooms_completo, dtype: int64


False    105370
True      15850
Name: rooms_completo, dtype: int64

## `SETEANDO RESTO DE ROOMS EN NULL`
ALGUNOS AJUSTES POSTERIORES ESPECÍFICOS PORQUE LA REGEX LOS TOMÓ Y NO APLICABA


AQUÍ SETEO EL RESTO QUE SE ENCUENTRA EN NULL

In [556]:
def fun_back(idreg):
    data.loc[idreg:idreg,"rooms_completo"]=data.loc[idreg:idreg,"rooms"]
    return data.loc[idreg:idreg,"rooms_completo"]
fun_back(53793)
fun_back(23884)

23884   NaN
Name: rooms_completo, dtype: float64

In [546]:
data["rooms_completo"].fillna(0,inplace = True) 
print(data["rooms_completo"].isnull().value_counts())

False    121220
Name: rooms_completo, dtype: int64


## `AGREGANDO CAMPOS NUEVOS`
Se crean nuevos campos para definir características que pueden revalorizar la propiedad
____

In [ ]:
data_encontrada1= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(pileta|piscina|pisina|picina)")
encontradas_pileta = data_encontrada1[data_encontrada1.notnull()].apply(lambda x: x.group(0))
data["PILETA"] = encontradas_pileta.apply(lambda x: False if x.count("sin")>0 else True)
data["PILETA"].fillna(value=False, inplace=True)
data["PILETA"].value_counts()

In [ ]:
data_encontrada_parrilla= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(parrilla|asado|grill|churrasquera)")
encontradas_parrilla = data_encontrada_parrilla[data_encontrada_parrilla.notnull()].apply(lambda x: x.group(0))
data["PARRILLA"] = encontradas_parrilla.apply(lambda x: False if x.count("sin")>0 else True)
data["PARRILLA"].fillna(value=False, inplace=True)
# data["PARRILLA"]= data_encontrada_parrilla.notnull()
data["PARRILLA"].value_counts()

In [ ]:
data_encontrada_patio= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(patio|jardin)")
encontradas_patio = data_encontrada_patio[data_encontrada_patio.notnull()].apply(lambda x: x.group(0))
data["PATIO_JARDIN"] = encontradas_patio.apply(lambda x: False if x.count("sin")>0 else True)
data["PATIO_JARDIN"].fillna(value=False, inplace=True)
data["PATIO_JARDIN"].value_counts()

ESTO ES PARA REALIZAR VALIDACIÓN DE DATOS


______


In [558]:
# codigos para ser leidos por cantidad de palabras habitación, se setean en NaN
#112626, 
#fun_back(69209)
idreg= 23884
print(data.title[idreg])
display(data.loc[idreg:idreg,["property_type","regex","rooms","rooms_completo","description"]])
display(data_room_null_description[idreg])
data_finded= fun_buscar_patt(data["description"][data["rooms_completo"].isnull()],"r(dormitorio)")
print("--___----")
display(data_finded[data_finded.notnull()])

Almagro Venta Casa de 10 ambientes con terraza.


,property_type,regex,rooms,rooms_completo,description
23884,house,([\s]\d?\d)[\s]?amb,NaN,9.0,CODIGO: 1259-Dr J Valentin Gomez 3500 ubicado ...


'codigo: 1259-dr j valentin gomez 3500 ubicado en: dr j gomez al 3500 -  publicado por: mb inmobiliaria s.a.. el precio es de usd 385000 null. observaciones:  almagro venta casa de 10 ambientes con terraza. dirección: valentín gómez al 3500  casa de 10 ambientes al frente distribuido en tres plantas. cada planta corresponde a un departamento, se puede utilizar en conjunto o por separado. cuenta con living comedor, cocina, comedor diario, dependencia, suite, hall, lavadero, toilette, vestidor, escritorio, sala de juegos, biblioteca, altillo,  baulera, 4 baños y un baño de servicio. terraza propia con parrilla de 8.60 x 6.50. ideal hostel, emprendimiento gastronomico, uso comercial. lote 8.60x18.49  r2aii  próximo a av. corrientes, av. díaz vélez y a estación de subte línea "b".  consultas al 11-43014375/11-54022048 o vía mail a  mb inmobiliaria s.a. irala 1648 caba     **la oferta de este inmueble es a sólo título informativo, pues la venta de esta propiedad está supeditada a que el pro

--___----


Series([], Name: description, dtype: object)

In [ ]:

# encontradas
# mask
#data.loc[data['Unnamed: 0'] == 53793, 'room_completo'] = 0
# data["rooms_completo"][mask] = encontradas
# room_null_mask = data["rooms_completo"].isnull()
# data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
# # print("Disminución en función a los ambientes tipo:",patt)
# # result = data["rooms_completo"].isnull().value_counts()

In [ ]:
data.dtypes

In [547]:
data["rooms_completo"][data.rooms.isnull()].sort_values(ascending=False)

72177     20.0
48620     14.0
16891     13.0
23844     12.0
112867    11.0
          ... 
93229      0.0
60875      0.0
121219     0.0
53793     -1.0
23884     -1.0
Name: rooms_completo, Length: 73830, dtype: float64

In [ ]:
data.pivot_table('rooms', index='rooms_completo', columns='property_type', aggfunc=np.mean )

In [ ]:
print(resultado[resultado.notnull()].value_counts())

In [ ]:
#display(data.loc[:,["property_type","rooms_completo","description"]])
#data.pivot_table('survived', index='sex', columns='class', margins=True, margins_name= "totales")
#resultado = data.pivot_table(["rooms","rooms_completo"],index='rooms_completo', columns="property_type", margins=True)
vale = (data.loc[:,"rooms_completo"]) > 10
#data.groupby('BARRIO')['M2'].median().sort_values(ascending=False)
data.loc[vale,"rooms_completo"].sort_values(ascending=False)

In [ ]:

# aggfunc={'PRECIOTEXTM':[np.mean,np.std,len],
#                           'AMBIENTES': [np.mean,np.std]})

resultado = data[["property_type","rooms","rooms_completo"]]
resultado.groupby("property_type")["rooms"].mean()

In [ ]:
data.query(title like "%precio%")